# Lab 02: Handwritten Digit Recognition App

**Group 09** 

---

## 1. Introduction
This notebook serves as the final application interface. It allows users to:
1.  **Upload** an image of a handwritten digit (or use sample images).
2.  **Process** the image (grayscale, resize, feature extraction).
3.  **Predict** the digit using our best trained Softmax Regression model.
4.  **Visualize** the prediction confidence.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from io import BytesIO
from PIL import Image

# Re-define SoftmaxRegression Class to load weights
class SoftmaxRegression:
    def __init__(self, n_features, n_classes):
        self.W = None
        self.b = None

    def softmax(self, z):
        z_stable = z - np.max(z, axis=1, keepdims=True)
        exp_z = np.exp(z_stable)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def predict_proba(self, X):
        z = np.dot(X, self.W) + self.b
        return self.softmax(z)

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

    def load_weights(self, filepath):
        if not os.path.exists(filepath):
            print(f"Error: Weight file not found at {filepath}")
            return False
        data = np.load(filepath)
        self.W = data['W']
        self.b = data['b']
        print(f"Model loaded successfully from {filepath}")
        return True

print("Libraries imported and Model class defined.")

Libraries imported and Model class defined.


In [2]:
# 1. Initialize Model
# We use 784 features as our best model was likely trained on Raw Pixels
model = SoftmaxRegression(n_features=784, n_classes=10)

# 2. Load Weights
weight_path = "../models/best_model_weights.npz"
success = model.load_weights(weight_path)

if not success:
    print("Please run Notebook 2 to train and save the model first!")

Model loaded successfully from ../models/best_model_weights.npz


## 2. Image Preprocessing Pipeline

Real-world images are different from MNIST data. To get accurate predictions, we must preprocess user inputs to match the training data distribution:
1.  **Grayscale Conversion:** Convert RGB to 1 channel.
2.  **Inversion (Optional):** MNIST digits are white on black background. If user uploads black on white (typical for paper), we must invert colors.
3.  **Resizing:** Downscale to $28 \times 28$ pixels.
4.  **Normalization:** Scale pixel values to $[0, 1]$.
5.  **Flattening:** Reshape to $(1, 784)$.

In [3]:
def preprocess_image(image_pil):
    """
    Converts a PIL image to a format compatible with the model.
    """
    # 1. Convert to Grayscale
    img = image_pil.convert('L')
    
    # 2. Resize to 28x28
    img = img.resize((28, 28), Image.Resampling.LANCZOS)
    
    # 3. Convert to NumPy array
    img_array = np.array(img)
    
    # 4. Invert colors if necessary
    # MNIST is White text on Black background.
    # If the image is mostly bright (mean > 128), it's likely Black text on White bg.
    if np.mean(img_array) > 128:
        img_array = 255 - img_array
        
    # 5. Normalize to [0, 1]
    img_array = img_array.astype(np.float32) / 255.0
    
    # 6. Flatten
    img_flat = img_array.reshape(1, -1)
    
    return img_array, img_flat

## 3. Interactive Demo

Upload an image of a handwritten digit (0-9) to test the model.
*Note: For best results, use an image with a clear digit centered in the frame.*

In [5]:
# Create Widgets
uploader = widgets.FileUpload(
    accept='image/*',  # Accept all image formats
    multiple=False,
    description='Upload Image'
)

button_predict = widgets.Button(
    description='Predict',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check'
)

output_img = widgets.Output()
output_pred = widgets.Output()

def on_predict_clicked(b):
    # Clear previous outputs
    output_img.clear_output()
    output_pred.clear_output()
    
    if not uploader.value:
        with output_pred:
            print("Please upload an image first!")
        return

    # Get the uploaded file
    # ipywidgets 8.0+ changes how value is accessed
    try:
        content = uploader.value[0]['content'] # For new versions
    except:
        # Fallback for older versions or different dict structure
        # Sometimes it's a dict with filename as key
        key = list(uploader.value.keys())[0]
        content = uploader.value[key]['content']
        
    # Open image
    image = Image.open(BytesIO(content))
    
    # Preprocess
    img_28x28, img_flat = preprocess_image(image)
    
    # Display Original and Processed Image
    with output_img:
        fig, ax = plt.subplots(1, 2, figsize=(8, 4))
        ax[0].imshow(image)
        ax[0].set_title("Original Upload")
        ax[0].axis('off')
        
        ax[1].imshow(img_28x28, cmap='gray')
        ax[1].set_title("Processed (Model Input)")
        ax[1].axis('off')
        plt.show()
        
    # Predict
    probs = model.predict_proba(img_flat)[0]
    prediction = np.argmax(probs)
    confidence = probs[prediction]
    
    # Display Prediction Results
    with output_pred:
        print(f"\nModel Prediction: {prediction}")
        print(f"Confidence: {confidence*100:.2f}%")
        
        # Plot probability bar chart
        plt.figure(figsize=(8, 3))
        bars = plt.bar(range(10), probs, color='skyblue')
        bars[prediction].set_color('orange') # Highlight winner
        plt.xticks(range(10))
        plt.xlabel('Digit Class')
        plt.ylabel('Probability')
        plt.title('Prediction Probabilities')
        plt.ylim(0, 1)
        plt.show()

button_predict.on_click(on_predict_clicked)

# Layout
display(widgets.VBox([
    widgets.Label(value="Step 1: Upload a digit image"),
    uploader,
    widgets.Label(value="Step 2: Run Prediction"),
    button_predict,
    widgets.HBox([output_img, output_pred])
]))